# 导入相关函数库

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## load data

In [4]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


# reset default graph

In [17]:
tf.reset_default_graph()

## define placeholder

In [18]:
x = tf.placeholder(tf.float32,[None,28,28,1])
y = tf.placeholder(tf.float32,[None,10])

## define variables and compute processing

In [7]:
num_channels = 1

conv1_size = 3
conv1_deep = 32

conv2_size = 3
conv2_deep = 64

fc1_nodes = 128
num_classes = 10

with tf.variable_scope('layer1-conv1'):
    conv1_weights = tf.get_variable('weights',[conv1_size, conv1_size, num_channels, conv1_deep],
    initializer=tf.truncated_normal_initializer(stddev=0.1))
    conv1_biases = tf.get_variable('biases',[conv1_deep],initializer=tf.zeros_initializer)
    conv1 = tf.nn.conv2d(x,conv1_weights,strides=[1,2,2,1],padding='SAME')         ##############################################
    relu1 = tf.nn.relu(tf.nn.bias_add(conv1,conv1_biases))

with tf.variable_scope('layer2-pool1'):
    pool1 = tf.nn.max_pool(relu1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

with tf.variable_scope('layer3-conv2'):
    conv2_weights = tf.get_variable('weights',[conv2_size,conv2_size,conv1_deep,conv2_deep],
    initializer=tf.truncated_normal_initializer(stddev=0.1))
    conv2_biases = tf.get_variable('biases',[conv2_deep],initializer=tf.zeros_initializer)
    conv2 = tf.nn.conv2d(pool1,conv2_weights,strides=[1,2,2,1],padding='SAME')
    relu2 = tf.nn.relu(tf.nn.bias_add(conv2,conv2_biases))

with tf.variable_scope('layer4-pool2'):
    pool2 = tf.nn.max_pool(relu2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [8]:
pool2_shape = pool2.get_shape().as_list()
# from functools import reduce
# total_nodes = reduce(tf.multiply,pool2_shape[1:])
total_nodes = pool2_shape[1]*pool2_shape[2]*pool2_shape[3]
pool2_flat = tf.reshape(pool2,(-1,total_nodes))

In [9]:
pool2_shape

[None, 2, 2, 64]

In [22]:
# define a function to reshape raw data
def data_reshape(data):
    return np.reshape(data,[-1,28,28,1])

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
train_x, train_y = mnist.train.next_batch(100)
a,b = sess.run([pool2, pool2_flat],feed_dict={x:data_reshape(train_x)})

In [37]:
with tf.variable_scope('layer5-fc1'):
    fc1_weights = tf.get_variable('weights', [256, 128],initializer=tf.truncated_normal_initializer)
    fc1_biases = tf.get_variable('biases',[128],initializer=tf.zeros_initializer)

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [19]:
num_channels = 1

conv1_size = 3
conv1_deep = 32

conv2_size = 3
conv2_deep = 64

fc1_nodes = 128
num_classes = 10

with tf.variable_scope('layer1-conv1'):
    conv1_weights = tf.get_variable('weights',[conv1_size, conv1_size, num_channels, conv1_deep],
    initializer=tf.truncated_normal_initializer(stddev=0.1))
    conv1_biases = tf.get_variable('biases',[conv1_deep],initializer=tf.zeros_initializer)
    conv1 = tf.nn.conv2d(x,conv1_weights,strides=[1,2,2,1],padding='SAME')         ##############################################
    relu1 = tf.nn.relu(tf.nn.bias_add(conv1,conv1_biases))

with tf.variable_scope('layer2-pool1'):
    pool1 = tf.nn.max_pool(relu1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

with tf.variable_scope('layer3-conv2'):
    conv2_weights = tf.get_variable('weights',[conv2_size,conv2_size,conv1_deep,conv2_deep],
    initializer=tf.truncated_normal_initializer(stddev=0.1))
    conv2_biases = tf.get_variable('biases',[conv2_deep],initializer=tf.zeros_initializer)
    conv2 = tf.nn.conv2d(pool1,conv2_weights,strides=[1,2,2,1],padding='SAME')
    relu2 = tf.nn.relu(tf.nn.bias_add(conv2,conv2_biases))

with tf.variable_scope('layer4-pool2'):
    pool2 = tf.nn.max_pool(relu2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

pool2_shape = pool2.get_shape().as_list()
# from functools import reduce
# total_nodes = tf.map_fn(tf.multiply,pool2_shape[1:])
total_nodes = pool2_shape[1]*pool2_shape[2]*pool2_shape[3]
total_nodes = int(total_nodes)
pool2_flat = tf.reshape(pool2,(-1,total_nodes))

with tf.variable_scope('layer5-fc1'):
    fc1_weights = tf.get_variable('weights', [total_nodes, fc1_nodes],initializer=tf.truncated_normal_initializer(stddev=0.1))
    fc1_biases = tf.get_variable('biases',[fc1_nodes],initializer=tf.zeros_initializer)
    fc1 = tf.matmul(pool2_flat,fc1_weights)+fc1_biases
    relu3 = tf.nn.relu(fc1)

with tf.variable_scope('layer6-output'):
    output_weights = tf.get_variable('weights',[fc1_nodes,num_classes],initializer=tf.truncated_normal_initializer)
    output_biases = tf.get_variable('biases',[num_classes],initializer=tf.zeros_initializer)
    logit = tf.matmul(relu3,output_weights)+output_biases

## define loss function and optimizer

In [24]:
# inference
y_ = logit

# define loss function
scores = tf.nn.softmax(y_)
cross_entropy = -tf.reduce_sum(y*tf.log(tf.clip_by_value(scores,1e-30,1.0)),1)
loss = tf.reduce_mean(cross_entropy)

learning_rate = 0.01
global_step = tf.Variable(1,trainable=False)
optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate).minimize(loss,global_step=global_step)

In [25]:
# define a function to reshape raw data
def data_reshape(data):
    return np.reshape(data,[-1,28,28,1])

# initializing all variables and training model
batch_size = 300
training_epochs = 3000
display_epoch = 10

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(training_epochs):
        train_x, train_y = mnist.train.next_batch(batch_size)
        _,training_loss = sess.run([optimizer,loss],feed_dict={x:data_reshape(train_x),y:train_y})

        if (epoch+1) % display_epoch == 0:
            print('After {} epochs, loss on training data is {}'.format(epoch+1,training_loss))

After 10 epochs, loss on training data is 3.5264127254486084
After 20 epochs, loss on training data is 3.53157639503479
After 30 epochs, loss on training data is 3.555483102798462
After 40 epochs, loss on training data is 3.4738688468933105
After 50 epochs, loss on training data is 3.3952488899230957
After 60 epochs, loss on training data is 3.3361055850982666
After 70 epochs, loss on training data is 3.2544901371002197
After 80 epochs, loss on training data is 3.281383752822876
After 90 epochs, loss on training data is 3.409984827041626
After 100 epochs, loss on training data is 3.1839346885681152
After 110 epochs, loss on training data is 3.140779972076416
After 120 epochs, loss on training data is 3.1322970390319824
After 130 epochs, loss on training data is 2.9142425060272217
After 140 epochs, loss on training data is 2.999919891357422
After 150 epochs, loss on training data is 3.0238242149353027
After 160 epochs, loss on training data is 2.9819445610046387
After 170 epochs, loss o

After 1340 epochs, loss on training data is 1.6669485569000244
After 1350 epochs, loss on training data is 1.6188722848892212
After 1360 epochs, loss on training data is 1.6501352787017822
After 1370 epochs, loss on training data is 1.543147325515747
After 1380 epochs, loss on training data is 1.6404683589935303
After 1390 epochs, loss on training data is 1.5744253396987915
After 1400 epochs, loss on training data is 1.5849460363388062
After 1410 epochs, loss on training data is 1.546904444694519
After 1420 epochs, loss on training data is 1.6408504247665405
After 1430 epochs, loss on training data is 1.6216599941253662
After 1440 epochs, loss on training data is 1.603669285774231
After 1450 epochs, loss on training data is 1.5952630043029785
After 1460 epochs, loss on training data is 1.5951167345046997
After 1470 epochs, loss on training data is 1.5173288583755493
After 1480 epochs, loss on training data is 1.5389010906219482
After 1490 epochs, loss on training data is 1.582766056060

After 2650 epochs, loss on training data is 1.0149942636489868
After 2660 epochs, loss on training data is 1.0123474597930908
After 2670 epochs, loss on training data is 1.0233858823776245
After 2680 epochs, loss on training data is 0.9585253000259399
After 2690 epochs, loss on training data is 0.9694441556930542
After 2700 epochs, loss on training data is 1.0091521739959717
After 2710 epochs, loss on training data is 0.9783036112785339
After 2720 epochs, loss on training data is 1.042864441871643
After 2730 epochs, loss on training data is 1.0542993545532227
After 2740 epochs, loss on training data is 0.9993411302566528
After 2750 epochs, loss on training data is 0.9917824268341064
After 2760 epochs, loss on training data is 0.9093539714813232
After 2770 epochs, loss on training data is 0.9581583738327026
After 2780 epochs, loss on training data is 0.9498676657676697
After 2790 epochs, loss on training data is 0.9217634797096252
After 2800 epochs, loss on training data is 0.9241556525